In [1]:
pip install SpeechRecognition pyttsx3 pyaudio pydub  requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import speech_recognition as sr
import pyttsx3
import pyaudio
import requests
from datetime import datetime


In [3]:
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    
    try:
        print("Recognizing...")
        query = recognizer.recognize_google(audio, language="en")
        print(f"You said: {query}")
        return query.lower()
    except sr.UnknownValueError:
        print("Sorry, I couldn't understand what you said.")
        return ""
    except sr.RequestError as e:
        print(f"Sorry, there was an error with the request: {e}")
        return ""
    
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [4]:
def get_weather(city, lang="en"):
    api_key = '84667fdc469192a2df21abb77800c327'
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = f"{base_url}q={city}&appid={api_key}"
    response = requests.get(complete_url)
    data = response.json()
    
    if data["cod"] != "404":
        main = data["main"]
        weather = data["weather"][0]
        wind = data["wind"]
        sys = data["sys"]
        aqi_url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={data['coord']['lat']}&lon={data['coord']['lon']}&appid={api_key}"
        aqi_response = requests.get(aqi_url)
        aqi_data = aqi_response.json()
        
        temperature = main["temp"] - 273.15
        humidity = main["humidity"]
        wind_speed = wind["speed"]
        description = weather["description"]
        sunrise = sys["sunrise"]
        sunset = sys["sunset"]
        aqi = aqi_data["list"][0]["main"]["aqi"]

        return {
            "temperature": temperature,
            "humidity": humidity,
            "wind_speed": wind_speed,
            "description": description,
            "sunrise": sunrise,
            "sunset": sunset,
            "aqi": aqi
        }
    else:
        return "not found"
def get_health_recommendations(weather_info):
    recommendations = []
    
    if weather_info["temperature"] < 0:
        recommendations.append("It's very cold outside. Make sure to dress warmly.")
    elif weather_info["temperature"] > 30:
        recommendations.append("It's very hot outside. Stay hydrated and avoid prolonged exposure to the sun.")
    
    if weather_info["humidity"] > 70:
        recommendations.append("The humidity is high. It might feel hotter than the actual temperature.")
    
    if weather_info["aqi"] == 1:
        recommendations.append("The air quality is good. It's a great day to be outside.")
    elif weather_info["aqi"] == 2:
        recommendations.append("The air quality is fair. People who are sensitive to pollution should limit outdoor activities.")
    elif weather_info["aqi"] == 3:
        recommendations.append("The air quality is moderate. It's better to stay indoors if you have respiratory issues.")
    elif weather_info["aqi"] == 4:
        recommendations.append("The air quality is poor. Avoid outdoor activities and wear a mask if necessary.")
    elif weather_info["aqi"] == 5:
        recommendations.append("The air quality is very poor. Stay indoors and keep windows closed.")
    
    return " ".join(recommendations)


In [5]:
def main():
    speak("Hi! I'm your weather assistant. What location's weather do you want to know?")
    city = recognize_speech()
    if city:
        weather_info = get_weather(city)
        if weather_info == "not found":
            speak(f"Sorry, I couldn't find weather information for {city}.")
        else:
            weather_details = (
                f"The weather in {city} is {weather_info['description']} with a temperature of {weather_info['temperature']:.2f} degrees Celsius, "
                f"humidity of {weather_info['humidity']}%, wind speed of {weather_info['wind_speed']} meters per second, "
                f"sunrise at {weather_info['sunrise AM']} and sunset at {weather_info['sunset PM']}. "
                f"The air quality index is {weather_info['aqi']}."
            )
            speak(weather_details)
            health_recommendations = get_health_recommendations(weather_info)
            speak(health_recommendations)
    else:
        speak("Please try again.")

if __name__ == "__main__":
    main()

Listening...
Recognizing...
You said: Vadodara
